In [ ]:
import pandas as pd
import pickle
import numpy as np
import torch

from fnope.utils.misc import get_output_dir, read_pickle
from pathlib import Path


In [ ]:
out_dir = get_output_dir()


experiment_folder = out_dir / "linear_gaussian_experiment"
methods = [
    "FNOPE_equispaced_False",
    "FNOPE_always_equispaced_True",
    "spectral_NPE",
    "raw_FMPE",
    "spectral_FMPE"]
nsims = [100, 1_000, 10_000, 100_000]

import pandas as pd
lg_df = pd.DataFrame()
for method in methods:
    for nsim in nsims:
        for run in range(1,4):
            print(method,nsim,run)

            swd_res = read_pickle(experiment_folder / method / f"num_sim_{nsim}_run_{run}" / f"swd_results.pkl")
            swds = swd_res["swd"]
            tarp_res = read_pickle(experiment_folder / method / f"num_sim_{nsim}_run_{run}" / f"tarp_results.pkl")
            tarp_absolute_atcs = tarp_res["absolute_atcs"]
            sbc_res = read_pickle(experiment_folder / method / f"num_sim_{nsim}_run_{run}" / f"sbc_results.pkl")
            sbc_absolute_atcs = sbc_res["absolute_atcs"]
            predictive_mse_res = read_pickle(experiment_folder / method / f"num_sim_{nsim}_run_{run}" / f"predictive_check_results.pkl")
            predictive_mse = predictive_mse_res["mses"]
            random_seed_path = experiment_folder / method / f"num_sim_{nsim}_run_{run}" / f"random_seed.csv"
            random_seed = int(np.loadtxt(random_seed_path, delimiter=","))

            training_time_path = experiment_folder / method / f"num_sim_{nsim}_run_{run}" / "training_time.csv"
            training_time_csv = np.loadtxt(training_time_path, delimiter=",")
            num_epochs = training_time_csv[0]
            training_time = training_time_csv[1]  # in seconds
            time_per_epoch = training_time / num_epochs  # in seconds

            sampling_time_path = experiment_folder / method / f"num_sim_{nsim}_run_{run}" / "sampling_times.csv"
            sampling_times_df = pd.read_csv(sampling_time_path)
            print(sampling_times_df.head())
            sampling_times_df["time_per_sample"] = sampling_times_df["time_ns"]/sampling_times_df["num_samples"]
            sampling_times = sampling_times_df["time_per_sample"].mean()
            sampling_times_ms = sampling_times * 1e-6  # Convert from ns to milliseconds

            nn_param_path = experiment_folder / method / f"num_sim_{nsim}_run_{run}" / "nn_num_params.csv"
            nn_num_params = int(np.loadtxt(nn_param_path, delimiter=","))

            # print(random_seeds[method][nsim][run-1])
            # print(f"swd: {swds.shape}")
            # print(f"tarp: {tarp_absolute_atcs}")
            # print(f"sbc: {sbc_absolute_atcs.shape}")
            # print(f"predictive_mse: {predictive_mse.flatten().shape}")
            # print(f"random_seed: {random_seed}")
            # print(f"training_time: {training_time}")
            # print(f"sampling_time: {sampling_times_ms}")
            # print(f"nn_num_params: {nn_num_params}")


            lg_df = pd.concat([lg_df, pd.DataFrame({
                "method": method,
                "nsim": nsim,
                "random_seed": random_seed,
                "swds": [swds.cpu().numpy()],
                "tarps": tarp_absolute_atcs,
                "sbcs": [sbc_absolute_atcs.cpu().numpy()],
                "predictive_mses": [predictive_mse.cpu().numpy()],
                "training_epochs": num_epochs,
                "training_time_total": training_time,
                "training_time_per_epoch": time_per_epoch,
                "sampling_time_per_sample": sampling_times_ms,
                "nn_num_params": nn_num_params,
            },)], ignore_index=True)

lg_df.to_csv(experiment_folder / "summary.csv", index=False)

In [ ]:
experiment_folder = Path(out_dir/"sir_experiment" / "FNOPE")
methods = ["fno","simformer"]
nsims = [1_000, 10_000, 100_000]
sir_df = pd.DataFrame()
for method in methods:
    for nsim in nsims:
        for run in range(1,4):

            tarp_res = read_pickle(experiment_folder / f"num_sim_{nsim}_run_{run}" / f"{method}_tarp_results.pkl")
            tarp_absolute_atcs = tarp_res["absolute_atcs"]
            sbc_res = read_pickle(experiment_folder / f"num_sim_{nsim}_run_{run}"  / f"{method}_sbc_results.pkl")
            sbc_absolute_atcs = sbc_res["absolute_atcs"]
            predictive_mse_res = read_pickle(experiment_folder / f"num_sim_{nsim}_run_{run}" / f"{method}_predictive_check_results.pkl")
            predictive_mse = predictive_mse_res["mses"]
            print("predictive mse shape: ", predictive_mse.shape)
            random_seed_path = experiment_folder / f"num_sim_{nsim}_run_{run}" / f"random_seed.csv"
            random_seed = int(np.loadtxt(random_seed_path, delimiter=","))


            training_time_path = experiment_folder /  f"num_sim_{nsim}_run_{run}" / "training_time.csv"
            training_time_csv = np.loadtxt(training_time_path, delimiter=",")
            num_epochs = training_time_csv[0]
            training_time = training_time_csv[1]  # in seconds
            time_per_epoch = training_time / num_epochs  # in seconds

            sampling_time_path = experiment_folder /  f"num_sim_{nsim}_run_{run}" / "sampling_times.csv"
            sampling_times_df = pd.read_csv(sampling_time_path)
            print(sampling_times_df.head())
            sampling_times_df["time_per_sample"] = sampling_times_df["time_ns"]/sampling_times_df["num_samples"]
            sampling_times = sampling_times_df["time_per_sample"].mean()
            sampling_times_ms = sampling_times * 1e-6  # Convert from ns to milliseconds

            nn_param_path = experiment_folder / f"num_sim_{nsim}_run_{run}" / "nn_num_params.csv"
            nn_num_params = int(np.loadtxt(nn_param_path, delimiter=","))
            # print(random_seeds[method][nsim][run-1])
            # print(f"method: {method}, num_sim: {nsim}, run: {run}")
            # print(f"tarp: {tarp_absolute_atcs}")
            # print(f"sbc: {sbc_absolute_atcs.shape}")
            # print(f"predictive_mse: {predictive_mse.flatten().mean()}")
            # print(f"random_seed: {random_seed}")
            # print(f"training_time: {training_time}")
            # print(f"sampling_time: {sampling_times_ms}")
            # print(f"nn_num_params: {nn_num_params}")

            sir_df = pd.concat([sir_df, pd.DataFrame({
                "method": method,
                "nsim": nsim,
                "random_seed": random_seed,
                "tarps": tarp_absolute_atcs,
                "sbcs": [sbc_absolute_atcs.cpu().numpy()],
                "predictive_mses": [predictive_mse.cpu().numpy()],
                "training_epochs": num_epochs,
                "training_time_total": training_time,
                "training_time_per_epoch": time_per_epoch,
                "sampling_time_per_sample": sampling_times_ms,
                "nn_num_params": nn_num_params,
            },)], ignore_index=True)


sir_df.to_pickle(experiment_folder / "summary.pkl")

In [ ]:
sir_df

In [ ]:
experiment_folder = out_dir / "darcy_experiment"
methods = [
    "FNOPE_always_equispaced_False",
    "FNOPE_always_equispaced_True",
    "spectral_NPE",
    "raw_FMPE",
    "spectral_FMPE"]

nsims = [100, 1_000, 3_000, 10_000]
param_dim = 129  # for the darcy experiment, the parameter dimension is 129


darcy_df = pd.DataFrame()
for method in methods:
    for nsim in nsims:
        for run in range(1,4):
            print(method,nsim,run)


            tarp_res = read_pickle(experiment_folder / method / f"num_sim_{nsim}_run_{run}" / f"tarp_results.pkl")
            tarp_absolute_atcs = tarp_res["absolute_atcs"]
            sbc_res = read_pickle(experiment_folder / method / f"num_sim_{nsim}_run_{run}" / f"sbc_results.pkl")
            sbc_absolute_atcs = sbc_res["absolute_atcs"]
            predictive_mse_res = read_pickle(experiment_folder / method / f"num_sim_{nsim}_run_{run}" / f"predictive_check_results.pkl")
            predictive_mse = predictive_mse_res["mses"]
            prior_log_prob_res = read_pickle(experiment_folder / method / f"num_sim_{nsim}_run_{run}" / f"prior_log_probs.pkl")
            prior_log_probs = (prior_log_prob_res["prior_log_probs"].view(-1)/(param_dim**2)).cpu().numpy()
            if method in ["FNOPE_always_equispaced_True","FNOPE_always_equispaced_False","raw_FMPE"]:
                posterior_log_prob_res = read_pickle(experiment_folder / method / f"num_sim_{nsim}_run_{run}" / f"posterior_log_probs.pkl")
                posterior_log_probs = (posterior_log_prob_res["posterior_log_probs"]/(param_dim**2)).cpu().numpy()
            else:
                posterior_log_probs = np.nan
            random_seed_path = experiment_folder / method / f"num_sim_{nsim}_run_{run}" / f"random_seed.csv"
            random_seed = int(np.loadtxt(random_seed_path, delimiter=","))
            # print(random_seeds[method][nsim][run-1])

            training_time_path = experiment_folder / method / f"num_sim_{nsim}_run_{run}" / "training_time.csv"
            training_time_csv = np.loadtxt(training_time_path, delimiter=",")
            num_epochs = training_time_csv[0]
            training_time = training_time_csv[1]  # in seconds
            time_per_epoch = training_time / num_epochs  # in seconds

            sampling_time_path = experiment_folder / method / f"num_sim_{nsim}_run_{run}" / "sampling_times.csv"
            sampling_times_df = pd.read_csv(sampling_time_path)
            print(sampling_times_df.head())
            sampling_times_df["time_per_sample"] = sampling_times_df["time_ns"]/sampling_times_df["num_samples"]
            sampling_times = sampling_times_df["time_per_sample"].mean()
            sampling_times_ms = sampling_times * 1e-6  # Convert from ns to milliseconds

            nn_param_path = experiment_folder / method / f"num_sim_{nsim}_run_{run}" / "nn_num_params.csv"
            nn_num_params = int(np.loadtxt(nn_param_path, delimiter=","))

            # print(f"tarp: {tarp_absolute_atcs}")
            # print(f"sbc: {sbc_absolute_atcs.shape}")
            # print(f"predictive_mse: {predictive_mse.flatten().shape}")
            # print(f"random_seed: {random_seed}")
            # print(f"prior_log_probs: {prior_log_probs.shape}")
            # print(f"posterior_log_probs: {posterior_log_probs}")
            # print(f"training_time: {training_time}")
            # print(f"sampling_time: {sampling_times_ms}")
            # print(f"nn_num_params: {nn_num_params}")


            darcy_df = pd.concat([darcy_df, pd.DataFrame({
                "method": method,
                "nsim": nsim,
                "random_seed": random_seed,
                "tarps": tarp_absolute_atcs,
                "sbcs": [sbc_absolute_atcs.cpu().numpy()],
                "predictive_mses": [predictive_mse.cpu().numpy()],
                "prior_log_probs": [prior_log_probs],
                "posterior_log_probs": [posterior_log_probs],
                "training_epochs": num_epochs,
                "training_time_total": training_time,
                "training_time_per_epoch": time_per_epoch,
                "sampling_time_per_sample": sampling_times_ms,
                "nn_num_params": nn_num_params,
            },)], ignore_index=True)

darcy_df.to_csv(experiment_folder / "summary.csv", index=False)
darcy_df.to_pickle(experiment_folder / "summary.pkl")

In [ ]:
darcy_df

In [ ]:
# Ice Results go here

experiment_folder = out_dir / "ice_experiment"
methods = [
    "FNOPE_always_equispaced_False",
    "spectral_NPE",
    "raw_FMPE",
    "spectral_FMPE",
    "raw_NPE"]

nsims = [100, 1_000, 10_000, 100_000]
x_size = 441


ice_df = pd.DataFrame()
for method in methods:
    for nsim in nsims:
        for run in range(1,4):
            print(method,nsim,run)

            tarp_res = read_pickle(experiment_folder / method / f"num_sim_{nsim}_run_{run}" / f"tarp_results.pkl")
            tarp_absolute_atcs = tarp_res["absolute_atcs"]
            sbc_res = read_pickle(experiment_folder / method / f"num_sim_{nsim}_run_{run}" / f"sbc_results.pkl")
            sbc_absolute_atcs = sbc_res["absolute_atcs"]
            predictive_mse_res = read_pickle(experiment_folder / method / f"num_sim_{nsim}_run_{run}" / f"predictive_check_results.pkl")
            predictive_mse = predictive_mse_res["mses"]/x_size
            predictive_mse_real_data_res = read_pickle(experiment_folder / method / f"num_sim_{nsim}_run_{run}" / f"real_layers_predictive_check_results.pkl")
            predictive_mse_real_data = predictive_mse_real_data_res["mses"]
            random_seed_path = experiment_folder / method / f"num_sim_{nsim}_run_{run}" / f"random_seed.csv"
            random_seed = int(np.loadtxt(random_seed_path, delimiter=","))
            # print(random_seeds[method][nsim][run-1])

            training_time_path = experiment_folder / method / f"num_sim_{nsim}_run_{run}" / "training_time.csv"
            training_time_csv = np.loadtxt(training_time_path, delimiter=",")
            num_epochs = training_time_csv[0]
            training_time = training_time_csv[1]  # in seconds
            time_per_epoch = training_time / num_epochs  # in seconds

            sampling_time_path = experiment_folder / method / f"num_sim_{nsim}_run_{run}" / "sampling_times.csv"
            sampling_times_df = pd.read_csv(sampling_time_path)
            print(sampling_times_df.head())
            sampling_times_df["time_per_sample"] = sampling_times_df["time_ns"]/sampling_times_df["num_samples"]
            sampling_times = sampling_times_df["time_per_sample"].mean()
            sampling_times_ms = sampling_times * 1e-6  # Convert from ns to milliseconds

            nn_param_path = experiment_folder / method / f"num_sim_{nsim}_run_{run}" / "nn_num_params.csv"
            nn_num_params = int(np.loadtxt(nn_param_path, delimiter=","))


            # print(f"tarp: {tarp_absolute_atcs}")
            # print(f"sbc: {sbc_absolute_atcs.shape}")
            # print(f"predictive_mse: {predictive_mse.flatten().shape}")
            # print(f"random_seed: {random_seed}")
            # print(f"predictive_mse_real_data: {predictive_mse_real_data.flatten().shape}")
            # print(f"training_time: {training_time}")
            # print(f"sampling_time: {sampling_times_ms}")
            # print(f"nn_num_params: {nn_num_params}")

            ice_df = pd.concat([ice_df, pd.DataFrame({
                "method": method,
                "nsim": nsim,
                "random_seed": random_seed,
                "tarps": tarp_absolute_atcs,
                "sbcs": [sbc_absolute_atcs.cpu().numpy()],
                "predictive_mses": [predictive_mse.cpu().numpy()],
                "predictive_mses_real_data": [predictive_mse_real_data.cpu().numpy()],
                "training_epochs": num_epochs,
                "training_time_total": training_time,
                "training_time_per_epoch": time_per_epoch,
                "sampling_time_per_sample": sampling_times_ms,
                "nn_num_params": nn_num_params
            },)], ignore_index=True)

ice_df.to_csv(experiment_folder / "summary.csv", index=False)
ice_df.to_pickle(experiment_folder / "summary.pkl")

In [ ]:
ice_df

# Combine to generate markdown table of compute times

In [ ]:
methods = {"FNOPE_always_equispaced_False": "FNOPE",
           "FNOPE_always_equispaced_True": "FNOPE (fix)",
           "spectral_NPE": "Spectral NPE",
           "raw_FMPE": "Raw FMPE",
           "spectral_FMPE": "Spectral FMPE",
           "raw_NPE": "Raw NPE",
        #    "Simformer": "Simformer"
           }

task_names = {"Gaussian_Linear": lg_df,
            #   "SIRD": sir_df,
              "Darcy": darcy_df,
              "ice": ice_df}

which_nsim = 10_000

# Separate dictionaries for each metric per task


nn_params = {}
training_total = {}
training_per_epoch = {}
sampling_per_sample = {}

# Error dictionaries
training_total_err = {}
training_per_epoch_err = {}
sampling_per_sample_err = {}

for task,df in task_names.items():
    nn_params[task] = {}
    training_total[task] = {}
    training_per_epoch[task] = {}
    sampling_per_sample[task] = {}

    # Error dictionaries
    training_total_err[task] = {}
    training_per_epoch_err[task] = {}
    sampling_per_sample_err[task] = {}

    for method in methods.keys():
        df_view = df[(df["method"] == method) & (df["nsim"] == which_nsim)]
        if df_view.empty:
            nn_params[task][method] = np.nan
            training_total[task][method] = np.nan
            training_per_epoch[task][method] = np.nan
            sampling_per_sample[task][method] = np.nan
            training_total_err[task][method] = np.nan
            training_per_epoch_err[task][method] = np.nan
            sampling_per_sample_err[task][method] = np.nan
        else:
            nn_params[task][method] = int(df_view["nn_num_params"].values[0])
            training_total[task][method] = df_view["training_time_total"].values.mean()
            training_per_epoch[task][method] = df_view["training_time_per_epoch"].values.mean()
            sampling_per_sample[task][method] = df_view["sampling_time_per_sample"].values.mean()
            training_total_err[task][method] = df_view["training_time_total"].values.std()
            training_per_epoch_err[task][method] = df_view["training_time_per_epoch"].values.std()
            sampling_per_sample_err[task][method] = df_view["sampling_time_per_sample"].values.std()


print("training_time_total:", training_total)
print("training_time_per_epoch:", training_per_epoch)
print("nn_params:", nn_params)
# Build the dataframe
table_df = pd.DataFrame({("method", ""): methods.values()})

# Helper to format ± values
def format_with_uncertainty(mean, std):
    if pd.isna(mean):
        return "nan"
    if mean > 1000:
        return f"{mean:.2e} ± {std:.2e}"
    else:
        return f"{mean:.2f} ± {std:.2f}"

for task in task_names:
    table_df[(task, "# weights")] = nn_params[task].values()
    table_df[(task, "training (Tot.). [s]")] = [
        format_with_uncertainty(v, e)
        for v, e in zip(training_total[task].values(), training_total_err[task].values())
    ]
    table_df[(task, "training (/epoch) [s]")] = [
        format_with_uncertainty(v, e)
        for v, e in zip(training_per_epoch[task].values(), training_per_epoch_err[task].values())
    ]
    table_df[(task, "sampling time [ms]")] = [
        format_with_uncertainty(v, e)
        for v, e in zip(sampling_per_sample[task].values(), sampling_per_sample_err[task].values())
    ]

# MultiIndex for headers
table_df.columns = pd.MultiIndex.from_tuples(table_df.columns)


In [ ]:
# Transpose the DataFrame
table_df_transposed = table_df.T

# If you want to keep the MultiIndex columns, transpose will convert columns to rows and vice versa.
# Then generate the markdown:
# Print markdown without column labels
# print(table_df_transposed.to_markdown(headers=[]))

def transpose_to_markdown(
    df: pd.DataFrame,
    method_column_name=("method", ""),
    param_metric="# weights"
) -> str:
    if not isinstance(df.columns, pd.MultiIndex):
        raise ValueError("Expected MultiIndex columns")

    # Step 1: Set method as index
    if method_column_name in df.columns:
        df = df.set_index(method_column_name)
    else:
        raise ValueError(f"'{method_column_name}' not found in columns")

    # Step 2: Keep only data columns
    data = df.loc[:, df.columns.get_level_values(0) != method_column_name[0]]
    df_t = data.T  # Transpose

    # Step 3: Header
    methods = df.index.tolist()
    header = ['Task', 'Metric'] + [str(m) for m in methods]
    separator = ['---'] * len(header)

    lines = [
        '| ' + ' | '.join(header) + ' |',
        '| ' + ' | '.join(separator) + ' |'
    ]

    last_task = None
    for (task, metric) in df_t.index:
        if task != last_task and last_task is not None:
            lines.append('| ' + ' | '.join([''] * len(header)) + ' |')

        row = [str(task), str(metric)]
        for m in methods:
            value = df_t.loc[(task, metric), m]
            if metric == param_metric:
                if pd.isna(value):
                    row.append("nan")
                else:
                    row.append(f"{int(value):,}")
            else:
                row.append(str(value))
        lines.append('| ' + ' | '.join(row) + ' |')
        last_task = task

    return '\n'.join(lines)

print(transpose_to_markdown(table_df))

| Task | Metric | FNOPE | FNOPE (fix) | Spectral NPE | Raw FMPE | Spectral FMPE | Raw NPE |
| --- | --- | --- | --- | --- | --- | --- | --- |
| Gaussian_Linear | # weights | 109,797 | 108,581 | 566,590 | 299,310 | 86,302 | nan |
| Gaussian_Linear | training (Tot.). [s] | 187.29 ± 43.20 | 118.53 ± 9.05 | 405.94 ± 35.88 | 308.39 ± 1.56 | 147.87 ± 7.21 | nan |
| Gaussian_Linear | training (/epoch) [s] | 0.99 ± 0.09 | 0.87 ± 0.08 | 2.17 ± 0.05 | 0.31 ± 0.00 | 0.24 ± 0.00 | nan |
| Gaussian_Linear | sampling time [ms] | 2.16 ± 0.12 | 1.06 ± 0.10 | 0.05 ± 0.00 | 0.20 ± 0.03 | 0.24 ± 0.02 | nan |
|  |  |  |  |  |  |  |  |
| Darcy | # weights | 11,555,673 | 11,552,521 | 3,537,566 | 9,180,033 | 897,664 | nan |
| Darcy | training (Tot.). [s] | 4.33e+03 ± 1.33e+03 | 2.51e+03 ± 4.26e+01 | 1.23e+03 ± 3.41e+02 | 732.19 ± 5.31 | 656.72 ± 1.53 | nan |
| Darcy | training (/epoch) [s] | 20.30 ± 0.63 | 26.23 ± 0.12 | 2.82 ± 0.05 | 0.73 ± 0.01 | 0.66 ± 0.00 | nan |
| Darcy | sampling time [ms] | 279.55 ± 6.64 | 35.20 ± 3.58 | 0.21 ± 0.01 | 2.70 ± 0.10 | 1.95 ± 0.07 | nan |
|  |  |  |  |  |  |  |  |
| ice | # weights | 25,317 | nan | 235,608 | 96,210 | 92,340 | 360,933 |
| ice | training (Tot.). [s] | 2.84e+03 ± 8.67e+02 | nan | 1.18e+03 ± 1.64e+02 | 2.01e+03 ± 2.93e+01 | 1.90e+03 ± 9.84e+01 | 407.97 ± 13.39 |
| ice | training (/epoch) [s] | 9.82 ± 0.12 | nan | 4.21 ± 0.37 | 2.02 ± 0.04 | 1.92 ± 0.07 | 5.64 ± 0.07 |
| ice | sampling time [ms] | 22.79 ± 2.21 | nan | 1.50 ± 0.15 | 23.85 ± 0.63 | 16.61 ± 1.30 | 1.53 ± 0.06 |